In [1]:
import json
with open("../recipe1M_layers/layer3.json", "r") as f:
    dataset = json.load(f)


In [2]:
j_f = []
for rec in dataset:
    vi = []
    for i in range(len(rec["ingredients"])):
        if rec["valid"][i]:
            if rec["ingredients"][i]["text"] not in vi:
                vi.append(rec["ingredients"][i]["text"])
    xx = {"id": rec["id"], "ingredients": vi}
    j_f.append(xx)

In [3]:
ingredients_per_recipe = [x['ingredients'] for x in j_f]

In [4]:
ingredients_per_recipe = [' '.join([''.join(ingredient.split()) for ingredient in recipe]) for recipe in ingredients_per_recipe]

In [5]:
ingredients_per_recipe[0]

'penne cheesesauce cheddarcheese gruyerecheese driedchipotlepowder unsaltedbutter all-purposeflour milk koshersalt garlicpowder'

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [8]:
corpus = ingredients_per_recipe
X = vectorizer.fit_transform(corpus)

In [11]:
type(X)

scipy.sparse.csr.csr_matrix

In [9]:
corpus_len = len(corpus)

In [12]:
from scipy.sparse import find

In [14]:
import numpy as np
n = 20
my_ingredients_string = "pepperoni mozzarellacheese flour eggs water salt oliveoil fish chips chickenliver cocoapowder sugar oregano pistachio"
my_ingredients_list = set(my_ingredients_string.split(" "))
my_vector = vectorizer.transform([my_ingredients_string])[0]
scalar_products = X.multiply(my_vector).sum(1)
knn = np.squeeze(np.asarray(scalar_products.reshape((corpus_len)))).argsort()[-n:]
all_overlapping = {}
for nearest in knn:
    found_ingredients_indices = find(X[nearest])[1]
    for index in found_ingredients_indices:
        ingredient = vectorizer.get_feature_names()[index]
        all_overlapping[ingredient] = all_overlapping.get(ingredient, 0) + 1
index_most_occurrences = np.array(list(all_overlapping.values())).argsort()[-len(my_ingredients_list)*2:]
elements_most_occurrences = np.array(list(all_overlapping.keys()))[index_most_occurrences]
# print(elements_most_occurrences)
all_knn_ingredients = set(elements_most_occurrences)
best_ingredients = my_ingredients_list.intersection(all_knn_ingredients)
print(best_ingredients)
print(f'Number of starting ingredients: {len(my_ingredients_list)}')
print(f'Number of chosen ingredients: {len(best_ingredients)}')

{'oregano', 'flour', 'sugar', 'water', 'salt', 'cocoapowder', 'mozzarellacheese', 'pepperoni', 'oliveoil', 'eggs'}
Number of starting ingredients: 14
Number of chosen ingredients: 10
